In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import ray
ray.shutdown()

import joblib
from ray.util.joblib import register_ray

register_ray()
ray.init(address="ray://localhost:10001",
         runtime_env={"pip": ["scikit-learn"]})
                      #"working_dir": "./"})

#import mlflow
#mlflow.set_tracking_uri('http://127.0.0.1:5001')
# mlflow.autolog() #log_models=False, exclusive=True)
import sklearn
#mlflow.sklearn.autolog()#log_models=True)

digits = load_digits()

param_space = {
    'C': np.logspace(-6, 6, 30),
    'gamma': np.logspace(-8, 8, 30),
    'tol': np.logspace(-4, -1, 30),
    'class_weight': [None, 'balanced'],
}
model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=20, n_iter=10, verbose=10)

with joblib.parallel_backend('ray'):
    search.fit(digits.data, digits.target)


/emr/notebook-env/lib/python3.7/site-packages/ray/util/client/worker.py:258: UserWarning: Ray Client connection timed out. Ensure that the Ray Client port on the head node is reachable from your local machine. See https://docs.ray.io/en/latest/cluster/ray-client.html#step-2-check-ports for more information.
  "Ray Client connection timed out. Ensure that "


ConnectionError: ray client connection timeout